## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-10-03-18-46-42 +0000,nypost,Heartless thief robs 10-year-old of piggy bank...,https://nypost.com/2025/10/03/us-news/10-year-...
1,2025-10-03-18-40-18 +0000,nyt,How an Attack on a Synagogue in Manchester Unf...,https://www.nytimes.com/2025/10/03/world/europ...
2,2025-10-03-18-39-42 +0000,nyt,"U.S. Military Attacked Boat Off Venezuela, Kil...",https://www.nytimes.com/2025/10/03/us/politics...
3,2025-10-03-18-34-09 +0000,nyt,French Photojournalist Killed in Drone Attack ...,https://www.nytimes.com/2025/10/03/world/europ...
4,2025-10-03-18-32-04 +0000,latimes,"Two men accused of rigging poker games, cheati...",https://www.latimes.com/california/story/2025-...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2445/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
47,trump,44
104,shutdown,19
15,manchester,19
14,synagogue,19
13,attack,19


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
12,2025-10-03-18-18-00 +0000,wsj,The Trump administration has frozen $2.1 billi...,https://www.wsj.com/politics/policy/trump-admi...,113
24,2025-10-03-17-57-00 +0000,wsj,Trump ordered another lethal military strike o...,https://www.wsj.com/politics/national-security...,99
233,2025-10-03-01-13-09 +0000,nyt,Trump Escalates Attacks on Democrats in a Gove...,https://www.nytimes.com/2025/10/02/us/politics...,95
9,2025-10-03-18-23-42 +0000,nypost,Terrorist Jihad Al-Shamie was on bail for rape...,https://nypost.com/2025/10/03/world-news/uk-te...,95
205,2025-10-03-07-13-06 +0000,nypost,UK police name 2 victims of Manchester synagog...,https://nypost.com/2025/10/03/world-news/uk-po...,91


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
12,113,2025-10-03-18-18-00 +0000,wsj,The Trump administration has frozen $2.1 billi...,https://www.wsj.com/politics/policy/trump-admi...
9,95,2025-10-03-18-23-42 +0000,nypost,Terrorist Jihad Al-Shamie was on bail for rape...,https://nypost.com/2025/10/03/world-news/uk-te...
232,51,2025-10-03-01-22-00 +0000,wsj,Senators on both sides of the aisle pressed th...,https://www.wsj.com/politics/national-security...
142,46,2025-10-03-12-00-08 +0000,wapo,One victim in Yom Kippur synagogue attack in E...,https://www.washingtonpost.com/world/2025/10/0...
124,45,2025-10-03-13-38-31 +0000,nypost,Goldman Sachs CEO David Solomon warns stock ma...,https://nypost.com/2025/10/03/business/goldman...
309,42,2025-10-02-20-08-12 +0000,nypost,Hamas military chief rejects Trump’s cease-fir...,https://nypost.com/2025/10/02/world-news/hamas...
14,40,2025-10-03-18-16-44 +0000,nypost,Lorde off Apple Music in Israel after yelling ...,https://nypost.com/2025/10/03/business/lorde-o...
60,37,2025-10-03-16-17-17 +0000,nypost,Throng of glitzed-up Taylor Swift fans grab ol...,https://nypost.com/2025/10/03/us-news/taylor-s...
157,37,2025-10-03-10-46-15 +0000,nypost,Sarah Mullally named the first female Archbish...,https://nypost.com/2025/10/03/world-news/churc...
276,36,2025-10-02-21-46-24 +0000,nyt,Des Moines Schools Superintendent Arrested by ...,https://www.nytimes.com/2025/10/02/us/des-moin...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
